In [1]:
import glob
import os
import pandas as pd
import streamlit as st
import random
import geopandas as gpd
import pyproj
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as offline
import plotly.express as px

In [89]:
sub_link = pd.read_csv("./resources/subareaID.txt", sep='\s+')
sub_link.rename({'pasture_name': 'Past_Name_'}, axis=1, inplace=True)
sub_link

,OPC_file_name,sub_ids,Past_Name_
0,RANGE_1_15E_TGM.OPC,1,15E
1,RANGE_2_24W_TGM.OPC,2,24W
2,RANGE_3_26E_TGM.OPC,3,26E
3,RANGE_4_25SE_TGM.OPC,4,25SE
4,RANGE_5_31E_TGM.OPC,5,31E
5,RANGE_6_19N_TGM.OPC,6,19N
6,RANGE_7_7NW_TGM.OPC,7,7NW
7,RANGE_8_5E_TGM.OPC,8,5E
8,RANGE_9_17N_TGM.OPC,9,17N
9,RANGE_10_20SE_TGM.OPC,10,20SE


In [90]:
shp = gpd.read_file("./resources/AGM_pastures.shp")
shp

,OBJECTID,Join_Count,TARGET_FID,Past_Name_,Ownership,Treatment,Shape_Leng,Shape_Area,geometry
0,7,1,6,25NW,ARS_owned_operated,AGM,5452.238777,1.302836e+06,"POLYGON ((522426.013 4516343.094, 522408.312 4..."
1,22,1,21,18S,ARS_owned_operated,AGM,4857.757709,1.319548e+06,"POLYGON ((525626.018 4520389.857, 525636.546 4..."
2,26,1,25,NH,ARS_owned_operated,AGM,5360.138858,1.229997e+06,"POLYGON ((520004.487 4521929.000, 520002.242 4..."
3,34,1,33,7SE,ARS_owned_operated,AGM,5440.046074,1.227656e+06,"POLYGON ((525614.295 4521784.500, 525619.418 4..."
4,49,1,48,31W,ARS_owned_operated,AGM,5059.512852,1.488702e+06,"POLYGON ((524899.812 4516327.000, 524981.438 4..."
5,54,1,53,21N,ARS_owned_operated,AGM,4756.874982,1.257200e+06,"POLYGON ((519167.859 4519542.250, 519181.506 4..."
6,55,1,54,17S,ARS_owned_operated,AGM,4785.000434,1.287060e+06,"POLYGON ((527222.562 4519537.000, 527208.500 4..."
7,57,1,56,8E,ARS_owned_operated,AGM,4766.958013,1.271737e+06,"POLYGON ((527222.938 4521386.500, 527214.119 4..."
8,66,1,65,20NW,ARS_owned_operated,AGM,5467.264149,1.285627e+06,"POLYGON ((527243.560 4519536.809, 525644.497 4..."
9,75,1,74,26W,ARS_owned_operated,AGM,3165.892719,6.262205e+05,"POLYGON ((520843.344 4516292.000, 520836.813 4..."


In [91]:
shp = shp.merge(sub_link, on='Past_Name_')
shp.centroid.x

0    522952.957611
1    524839.825086
2    525095.397799
3    524517.067573
4    518397.274735
5    526410.227559
6    526828.983018
7    526175.250183
8    521229.714652
dtype: float64

In [93]:
subs_list = shp.sub_ids

In [94]:
acy_df = pd.read_csv(
            "./resources/CONUNN_AGM.ACY",
            delim_whitespace=True,
            skiprows=8,
            # nrows=1,
            header=0,
            usecols=['SA#', 'YR', 'BIOM']
            )
subids = acy_df['SA#'].unique()

In [95]:
dff = pd.DataFrame()
for i in subids:
    sacy_df = acy_df.loc[acy_df['SA#'] == i]
    sacy_df = sacy_df.groupby('YR').sum()
    sacy_df.drop('SA#', axis=1, inplace=True)
    sacy_df.rename({'BIOM': i}, axis=1, inplace=True)
    # sacy_df.index = sacy_df['YR']
    dff = pd.concat([dff, sacy_df], axis=1)



In [96]:
dff = dff.T
dff.index.name = 'sub_ids'

In [97]:
dff.index

Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
            20],
           dtype='int64', name='sub_ids')

In [110]:
dff.columns.min()

2008

In [106]:
dff.min().min()

1.46

In [99]:
shp = shp.merge(dff, on='sub_ids')
shp

,OBJECTID,Join_Count,TARGET_FID,Past_Name_,Ownership,Treatment,Shape_Leng,Shape_Area,geometry,OPC_file_name,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,7,1,6,25NW,ARS_owned_operated,AGM,5452.238777,1.302836e+06,"POLYGON ((522426.013 4516343.094, 522408.312 4...",RANGE_14_25NW_AGM.OPC,...,5.75,5.53,5.47,6.00,6.04,5.99,5.72,5.31,5.39,4.92
1,22,1,21,18S,ARS_owned_operated,AGM,4857.757709,1.319548e+06,"POLYGON ((525626.018 4520389.857, 525636.546 4...",RANGE_16_18S_AGM.OPC,...,11.20,11.42,12.57,13.55,13.66,14.49,14.51,14.86,15.84,15.28
2,34,1,33,7SE,ARS_owned_operated,AGM,5440.046074,1.227656e+06,"POLYGON ((525614.295 4521784.500, 525619.418 4...",RANGE_17_7SE_AGM.OPC,...,9.36,9.53,10.67,11.99,12.60,12.94,13.66,13.98,14.76,14.69
3,49,1,48,31W,ARS_owned_operated,AGM,5059.512852,1.488702e+06,"POLYGON ((524899.812 4516327.000, 524981.438 4...",RANGE_15_31W_AGM.OPC,...,8.28,8.53,10.00,11.21,11.63,11.75,12.51,12.61,12.96,12.90
4,54,1,53,21N,ARS_owned_operated,AGM,4756.874982,1.257200e+06,"POLYGON ((519167.859 4519542.250, 519181.506 4...",RANGE_12_21N_AGM.OPC,...,8.78,8.65,8.86,10.34,9.99,10.28,11.00,11.50,11.62,11.43
5,55,1,54,17S,ARS_owned_operated,AGM,4785.000434,1.287060e+06,"POLYGON ((527222.562 4519537.000, 527208.500 4...",RANGE_19_17S_AGM.OPC,...,9.14,9.57,10.82,12.26,12.67,13.36,14.09,14.54,15.08,15.28
6,57,1,56,8E,ARS_owned_operated,AGM,4766.958013,1.271737e+06,"POLYGON ((527222.938 4521386.500, 527214.119 4...",RANGE_18_8E_AGM.OPC,...,10.66,11.01,12.69,14.08,14.87,15.38,16.27,16.52,17.26,17.10
7,66,1,65,20NW,ARS_owned_operated,AGM,5467.264149,1.285627e+06,"POLYGON ((527243.560 4519536.809, 525644.497 4...",RANGE_20_20NW_AGM.OPC,...,8.03,7.64,7.20,8.85,9.01,9.54,9.91,9.92,10.23,10.04
8,75,1,74,26W,ARS_owned_operated,AGM,3165.892719,6.262205e+05,"POLYGON ((520843.344 4516292.000, 520836.813 4...",RANGE_13_26W_AGM.OPC,...,6.66,6.17,6.03,7.31,6.90,6.70,6.42,6.22,6.29,6.06


In [100]:
shp['sub_ids']

0    14
1    16
2    17
3    15
4    12
5    19
6    18
7    20
8    13
Name: sub_ids, dtype: int64

In [102]:
shp.to_crs(pyproj.CRS.from_epsg(4326), inplace=True)
# subdf = subdf[['Subbasin', 'geometry']]
# subdf['geometry'] = subdf['geometry'].convex_hull

shp.index = shp.sub_ids 
mfig = px.choropleth(shp,
                geojson=shp.geometry,
                locations=shp.index,
                color=2015,
                # #    projection="mercator"
                # animation_frame="time",
                # range_color=(0, 12),
                )
# mfig.update_layout()
mfig.update_geos(fitbounds="locations", visible=False)
mfig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
mfig

In [104]:
shp.centroid.y

<ipython-input-104-49c29d5c1cd2>:1: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.




sub_ids
14    40.807340
16    40.830340
17    40.845847
15    40.790419
12    40.823086
19    40.830132
18    40.848361
20    40.821676
13    40.804765
dtype: float64